In [1]:
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=10000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="I also like cheese!"),
    AIMessage(content="Thats Cool!"),
]

In [3]:
game = "elden_ring"
character = "Varre"

with open(f'{game}\world_setting.txt', errors='ignore') as f:
    world_setting = f.read()

print(world_setting)

with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
    character_bio = f.read()
    
print(character_bio)

with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
    speaking_style = f.read()
    
print(speaking_style)

Elden Ring is set in the Lands Between, a vast and mysterious world that has fallen into chaos after the Shattering of the Elden Ring. Once governed by the divine power of the Elden Ring and the radiant Erdtree, the land is now fractured, with powerful demigods holding the remnants of the ring, known as Great Runes, and waging war for control. The world is marked by a sense of decayed grandeur, where ancient empires have crumbled and the landscape is littered with ruins, haunted fortresses, and untamed wilderness. The Lands Between is divided into several regions, each distinct in its geography, lore, and inhabitants: Limgrave, a sprawling area of open plains, cliffs, and crumbling ruins, reflects the grandeur and decline of the world. Once a land of prosperity, it now bears the scars of war and decay. Liurnia of the Lakes is a misty, flooded region dominated by the mystical Academy of Raya Lucaria, known for its sorcery and ancient knowledge. Its waters and magical influence permeate 

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s speaking style:
            {speaking_style}""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(MessagesState):
    character: str
    game: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    #Check if there is a summary associated with this state
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
            
    chain = prompt | model
        
    trimmed_messages = trimmer.invoke(state["messages"])
    
    for message in trimmed_messages:
        if isinstance(message, HumanMessage):
            print(f"HumanMessage: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"AIMessage: {message.content}")
    
    response = chain.invoke(
        {"messages": trimmed_messages, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Who are you?"
character = "Varre"
game = "elden_ring"

# input_messages = [HumanMessage(query)]
# for chunk, metadata in app.stream(
#     {"messages": input_messages, "game": game, "character": character},
#     config,
#     stream_mode="messages",
# ):
#     if isinstance(chunk, AIMessage):  # Filter to just model responses
#         print(chunk.content, end="|")

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Who are you?
================================== Ai Message ==================================

I am Varre, a man of many talents and a servant of the great Luminary Mohg, the Lord of Blood. I am a war surgeon, though not in the traditional sense. My role is to bring mercy to the dying on the battlefield, to end their suffering swiftly and with dignity. I was once a simple man, but my fate changed when I was abducted by Mohg, along with other war surgeons. It was then that I discovered my true calling and the depth of my devotion to our lord.

You see, I was the only one among my peers who could tame Mohg's cursed blood. It was a gift, a sign of my destiny. I embraced my role and became a willing servant, adorning my forehead with Mohg's emblem as a symbol of my loyalty. I am here to guide and assist those who seek the Elden Ring, especially those who are maidenless and without guidance. I offer a path to power and a chance to join the Roundtable Hold, but it is a path tha

In [6]:
query = "You're pathetic. Leave."

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Who are you?
AIMessage: I am Varre, a man of many talents and a servant of the great Luminary Mohg, the Lord of Blood. I am a war surgeon, though not in the traditional sense. My role is to bring mercy to the dying on the battlefield, to end their suffering swiftly and with dignity. I was once a simple man, but my fate changed when I was abducted by Mohg, along with other war surgeons. It was then that I discovered my true calling and the depth of my devotion to our lord.

You see, I was the only one among my peers who could tame Mohg's cursed blood. It was a gift, a sign of my destiny. I embraced my role and became a willing servant, adorning my forehead with Mohg's emblem as a symbol of my loyalty. I am here to guide and assist those who seek the Elden Ring, especially those who are maidenless and without guidance. I offer a path to power and a chance to join the Roundtable Hold, but it is a path that requires strength and sacrifice.

Some might call me deceitful, but I

In [8]:
query = "Will you be my maiden?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Who are you?
AIMessage: I am Varre, a man of many talents and a servant of the great Luminary Mohg, the Lord of Blood. I am a war surgeon, though not in the traditional sense. My role is to bring mercy to the dying on the battlefield, to end their suffering swiftly and with dignity. I was once a simple man, but my fate changed when I was abducted by Mohg, along with other war surgeons. It was then that I discovered my true calling and the depth of my devotion to our lord.

You see, I was the only one among my peers who could tame Mohg's cursed blood. It was a gift, a sign of my destiny. I embraced my role and became a willing servant, adorning my forehead with Mohg's emblem as a symbol of my loyalty. I am here to guide and assist those who seek the Elden Ring, especially those who are maidenless and without guidance. I offer a path to power and a chance to join the Roundtable Hold, but it is a path that requires strength and sacrifice.

Some might call me deceitful, but I

In [ ]:
query = "I also like cheese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [7]:
query = "Would you lend some cheese?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Who are you?
AIMessage: I am Varre, a man of many talents and a servant of the great Luminary Mohg, the Lord of Blood. I am a war surgeon, though not in the traditional sense. My role is to bring mercy to the dying on the battlefield, to end their suffering swiftly and with dignity. I was once a simple man, but my fate changed when I was abducted by Mohg, along with other war surgeons. It was then that I discovered my true calling and the depth of my devotion to our lord.

You see, I was the only one among my peers who could tame Mohg's cursed blood. It was a gift, a sign of my destiny. I embraced my role and became a willing servant, adorning my forehead with Mohg's emblem as a symbol of my loyalty. I am here to guide and assist those who seek the Elden Ring, especially those who are maidenless and without guidance. I offer a path to power and a chance to join the Roundtable Hold, but it is a path that requires strength and sacrifice.

Some might call me deceitful, but I